In [1]:
import numpy as np

In [2]:
def UF7(pop):
    fitness = np.empty((0,2))
    for x in pop:
        J1 = 0
        J2 = 0

        sum1 = 0
        sum2 = 0
        fit_ind = np.array([])
        
        for j in range(1, len(x)):
            if j % 2 == 0:
                J1 += 1
                sum1 += (x[j] - np.sin(6 * np.pi * x[0] + ((j+1) * np.pi) / len(x))) ** 2
            if j % 2 == 1:
                J2 += 1
                sum2 += (x[j] - np.sin(6 * np.pi * x[0] + ((j+1) * np.pi) / len(x))) ** 2

        if J1 == 0:
            f1 = x[0]
        else:
            f1 = x[0] + 2 / J1 * sum1

        if J2 == 0:
            f2 = 1 - np.sqrt(x[0])
        else:
            f2 = 1 - np.sqrt(x[0]) + 2 / J2 * sum2
        
        fit_ind = np.array([[f1, f2]])
        fitness = np.vstack((fitness, fit_ind))
    return fitness

In [3]:
"""
tau — пропорция нечувствительных точек в обучающем наборе.
phi — количество случайно выбранных индивидов.
wmax — количество поколений перед обновлением архивов (CA и DA).
mu — количество решений, которые переоценивались на каждом шаге.
"""
tau = 0.75
phi = 0.1
wmax = 10
mu = 5
left = np.array([0, -1, -1])
right = np.array([1, 1, 1])

In [4]:
def UniformPoint(N, D, method='Latin'):
    """
    N — количество индивидов.
    D — размерность пространства.
    method — метод генерации точек (опционально). Если метод не указан, используется метод по умолчанию.
    Используется для генерации набора равномерно распределенных точек в многомерном пространстве целей. 
    NBI, ILD, MUD, grid или Latin, будет выбрана в зависимости от значения method.
    """
    if method == 'Latin':
        return Latin(N, D)
    
def Latin(N, D):
    """
    W - индексная матрица, в которой элементы упорядочены по возрастанию, но содержат индексы исходной матрицы.
    """
    W = np.argsort(np.random.rand(N, D), axis=0)
    return (np.random.rand(N, D) + W) / N

In [5]:
def EvalFitness(P, l, r, function):
    """
    P - матрица индивидов.
    l, r - границы.
    function - целевая функция.
    """
    points = (r - l) * P + l
    return function(points)

In [6]:
def UpdateCA(CA, New, MaxSize):
    """
    CA - архив лучших.
    New - новые индивиды.
    MaxSize - максимальный размер архива.
    Обновляем архив, добавляя новые решения и выбирая индивидов с наилучшей пригодностью.
    """
    CA = np.vstack((CA, New))
    n = len(CA)
    if n <= MaxSize:
        return CA
    
    CAObj = CA[:, -M:]# пригодность индивидов
    
    CAmax = np.max(CAObj, axis=0)
    CAmin = np.min(CAObj, axis=0)
    
    #Нормализуем пригодность
    CAObj = (CAObj - CAmin) / (CAmax - CAmin)
    
    I = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            I[i, j] = np.max(CAObj[i, :] - CAObj[j, :])
            
    C = np.max(np.abs(I), axis=0)
    F = np.sum(-np.exp(-I / C / 0.05), axis=1) + 1
    
    Choose = list(range(n))
    
    # Удаляем решения с наименьшей пригодностью
    while len(Choose) > MaxSize:
        x = np.argmin(F[Choose])
        F += np.exp(-I[Choose[x], :] / C[Choose[x]] / 0.05)
        del Choose[x]
    
    # Возвращаем обновленный набор решений
    return CA[Choose]
    

In [7]:
pop = UniformPoint(10, 3)
pop

array([[0.44538108, 0.45698579, 0.51795889],
       [0.70956137, 0.04621937, 0.04166945],
       [0.2958842 , 0.76738912, 0.91729584],
       [0.17962529, 0.25736538, 0.80202049],
       [0.91100086, 0.36137904, 0.43100534],
       [0.07927551, 0.62821787, 0.29344374],
       [0.39027624, 0.16249593, 0.16267574],
       [0.51962044, 0.57660096, 0.62938084],
       [0.64464338, 0.84554967, 0.32948451],
       [0.87320034, 0.90926949, 0.78959734]])

In [8]:
left = np.array([0, -1, -1])
right = np.array([1, 1, 1])
fit = EvalFitness(pop, left, right, UF7)
fit

array([[2.04021167, 1.57669224],
       [0.7843508 , 2.77490573],
       [0.36498173, 0.85862295],
       [0.44200069, 0.68580899],
       [3.47517049, 0.97600043],
       [0.76130267, 1.66732554],
       [0.47363455, 1.2173182 ],
       [0.54071563, 1.49576781],
       [1.75272489, 0.38074975],
       [0.89463427, 2.5296141 ]])

In [9]:
Population = np.hstack((pop, fit))         
Population#индивиды + пригодности

array([[0.44538108, 0.45698579, 0.51795889, 2.04021167, 1.57669224],
       [0.70956137, 0.04621937, 0.04166945, 0.7843508 , 2.77490573],
       [0.2958842 , 0.76738912, 0.91729584, 0.36498173, 0.85862295],
       [0.17962529, 0.25736538, 0.80202049, 0.44200069, 0.68580899],
       [0.91100086, 0.36137904, 0.43100534, 3.47517049, 0.97600043],
       [0.07927551, 0.62821787, 0.29344374, 0.76130267, 1.66732554],
       [0.39027624, 0.16249593, 0.16267574, 0.47363455, 1.2173182 ],
       [0.51962044, 0.57660096, 0.62938084, 0.54071563, 1.49576781],
       [0.64464338, 0.84554967, 0.32948451, 1.75272489, 0.38074975],
       [0.87320034, 0.90926949, 0.78959734, 0.89463427, 2.5296141 ]])

In [83]:
M = 2
UpdateCA(np.empty((0,5)), Population, 5)

array([[0.28053072, 0.22616213, 0.68351992, 0.72588279, 4.60873815],
       [0.01282734, 0.92106056, 0.95231092, 2.63058554, 0.91602795],
       [0.87112897, 0.52819526, 0.08874818, 5.22933775, 0.36301364],
       [0.69256013, 0.34279513, 0.74352317, 2.52034599, 1.59533393],
       [0.9191105 , 0.61193708, 0.24171176, 5.51268601, 0.24034641]])

In [ ]:
def dacefit(S,Y,regr,corr,theta0,lob,upb):
    """
    S, Y - точки данных и значения функции в этих точках.
    regr - функция регрессии.
    corr - корреляционная функция.
    theta0 - параметры корреляционной функции.
    lob, upb - нижние и верхние границы для оптимизации параметров корреляции.
    Если присутствует, то нижняя и верхняя границы theta. В противном случае для theta используется theta0..
    """
    
    #Проверка корректности входных данных
    m,n = S.shape
    #Если вектор строчный, то преоброзуем его в стобец и сохраним длинну в lenY, иначе сохраним кол-во строк в lenY
    if Y.ndim == 1:
        Y = Y.reshape(-1,1)
        lenY = len(Y)
    else:
        lenY = Y.shape[0]
        
    if S.shape[0] != Y.shape[0]:
        raise ValueError("S и Y должнны иметь одинаковое количество строк")
        
    #Проверка параметров корреляции, если заданы границы
        
    